# Download NFIP Redacted Claims Data from OpenFEMA

In [1]:
# import libraries
import requests
from datetime import datetime
import duckdb

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p requests,duckdb

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

requests: 2.28.1
duckdb  : 1.0.0



In [3]:
# data retrieved
current_date = datetime.now()
print(f"The data was retrieved on {current_date.strftime('%Y-%m-%d')}.")

The data was retrieved on 2025-03-15.


# OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2

OpenFEMA NFIP Claims Dataset: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2

![dataset-page](images/dataset-page.png)

# Dataset Download Link
Right-click the *Link to parquet* address, and select *Copy Link Address*.

![dataset-page](images/download-link.png)

# Download Full Data as Parquet file

In [4]:
# URL of the Parquet file to download
url = "https://www.fema.gov/about/reports-and-data/openfema/FimaNfipClaims.parquet"

# specify the file path where you want to save the downloaded file
file_path = "data/FimaNfipClaims.parquet"

# send an HTTP GET request to the URL
response = requests.get(url)

# check if the request was successful (status code 200)
if response.status_code == 200:
    
    # write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.")

File downloaded successfully.


In [61]:
ls data/

FimaNfipClaims.parquet


In [62]:
# preview size of file
!du -sh data/* | sort -rh

151M	data/FimaNfipClaims.parquet


In [63]:
# create a DuckDB database instance
con = duckdb.connect()

# create table claims of dataset
con.execute("""
    CREATE TABLE claims AS
        FROM read_parquet('data/FimaNfipClaims.parquet')
""")

# sanity check
sql = """
    SELECT *
    FROM claims
    LIMIT 5
"""

con.sql(sql)

┌──────────────────────┬──────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ … │   latitude   │  longitude   │          id          │
│       boolean        │ timestamp with tim…  │   │ decimal(9,1) │ decimal(9,1) │         uuid         │
├──────────────────────┼──────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ false                │ 2020-12-11 11:25:4…  │ … │         30.3 │        -87.7 │ 11ccaba6-1fd2-45e7…  │
│ false                │ 2020-11-13 09:50:3…  │ … │         30.3 │        -87.7 │ b70a2032-8741-4995…  │
│ false                │ 2020-03-27 08:15:4…  │ … │         33.5 │       -112.1 │ 1d3954b2-a77f-49b4…  │
│ false                │ 2020-03-27 08:15:4…  │ … │         33.5 │       -112.1 │ 6782824b-6c87-4215…  │
│ false                │ 2020-03-26 08:56:2…  │ … │         34.4 │       -119.6 │ d8a6689f-3e3a-4256…  │
├──────────────────────┴──────────────────────┴───┴────

In [64]:
# sanity check
sql = """
    SELECT COUNT(*) AS count
    FROM claims
"""

con.sql(sql)

┌─────────┐
│  count  │
│  int64  │
├─────────┤
│ 2707173 │
└─────────┘

In [65]:
# sanity check
sql = """
    SELECT
        COUNT(column_name) AS count_columns
    FROM
        (DESCRIBE FROM claims)
"""

con.sql(sql)

┌───────────────┐
│ count_columns │
│     int64     │
├───────────────┤
│            73 │
└───────────────┘

In [66]:
# sanity check
sql = """
    SELECT
        asOfDate
    FROM
        claims
    ORDER BY
        asOfDate DESC
    LIMIT 1
"""

con.sql(sql)

┌───────────────────────────────┐
│           asOfDate            │
│   timestamp with time zone    │
├───────────────────────────────┤
│ 2025-03-10 12:21:38.590949-04 │
└───────────────────────────────┘

In [67]:
# sanity check
sql = """
    SELECT
        column_name,
        column_type
    FROM
        (DESCRIBE claims)
"""

con.sql(sql).show(max_rows=80)

┌────────────────────────────────────────────┬──────────────────────────┐
│                column_name                 │       column_type        │
│                  varchar                   │         varchar          │
├────────────────────────────────────────────┼──────────────────────────┤
│ agricultureStructureIndicator              │ BOOLEAN                  │
│ asOfDate                                   │ TIMESTAMP WITH TIME ZONE │
│ basementEnclosureCrawlspaceType            │ SMALLINT                 │
│ policyCount                                │ SMALLINT                 │
│ crsClassificationCode                      │ SMALLINT                 │
│ dateOfLoss                                 │ DATE                     │
│ elevatedBuildingIndicator                  │ BOOLEAN                  │
│ elevationCertificateIndicator              │ VARCHAR                  │
│ elevationDifference                        │ DECIMAL(6,1)             │
│ baseFloodElevation                  

In [68]:
# sanity check
sql = """
    SELECT
        column_name,
        null_percentage
    FROM
        (SUMMARIZE FROM claims)
    WHERE
        null_percentage > 0
    ORDER BY
        null_percentage DESC
"""

con.sql(sql).show(max_rows=80)

┌────────────────────────────────────────────┬─────────────────┐
│                column_name                 │ null_percentage │
│                  varchar                   │  decimal(9,2)   │
├────────────────────────────────────────────┼─────────────────┤
│ floodCharacteristicsIndicator              │           98.53 │
│ eventDesignationNumber                     │           93.23 │
│ lowestAdjacentGrade                        │           81.09 │
│ crsClassificationCode                      │           79.69 │
│ nonPaymentReasonBuilding                   │           77.94 │
│ elevationCertificateIndicator              │           77.91 │
│ lowestFloorElevation                       │           76.35 │
│ baseFloodElevation                         │           75.66 │
│ elevationDifference                        │           72.94 │
│ floodZoneCurrent                           │           71.75 │
│ nfipCommunityNumberCurrent                 │           71.70 │
│ basementEnclosureCrawls

# Retrieve NFIP Claims for New York City

The FIPS codes for New York City boroughs are:  
- Bronx: 36005
- Brooklyn: 36047
- Manhattan: 36061
- Queens: 36081
- Staten Island: 36085

In [69]:
# sanity check
sql = """
    SELECT countyCode
    FROM claims
    LIMIT 10
"""

con.sql(sql)

┌────────────┐
│ countyCode │
│  varchar   │
├────────────┤
│ 01003      │
│ 01003      │
│ 04013      │
│ 04013      │
│ 06083      │
│ 09001      │
│ 12113      │
│ 12021      │
│ 12113      │
│ 12086      │
├────────────┤
│  10 rows   │
└────────────┘

In [70]:
# sanity check
sql = """
    SELECT
        countyCode,
        COUNT(*) AS count
    FROM
        claims
    WHERE
        countyCode in ('36005', '36047', '36061', '36081', '36085')
    GROUP BY
        countyCode
    ORDER BY
        count DESC
        
"""

con.sql(sql)

┌────────────┬───────┐
│ countyCode │ count │
│  varchar   │ int64 │
├────────────┼───────┤
│ 36085      │ 19414 │
│ 36081      │ 14873 │
│ 36047      │  6264 │
│ 36005      │  1942 │
│ 36061      │  1485 │
└────────────┴───────┘

In [71]:
sql = """
    SELECT *
    FROM claims
    WHERE countyCode in (
        '36005',
        '36047',
        '36061',
        '36081',
        '36085'
    )    
"""

new_york_city = con.sql(sql)

new_york_city.show(max_rows=5)

┌──────────────────────┬──────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ … │   latitude   │  longitude   │          id          │
│       boolean        │ timestamp with tim…  │   │ decimal(9,1) │ decimal(9,1) │         uuid         │
├──────────────────────┼──────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ false                │ 2024-06-02 14:37:4…  │ … │         40.6 │        -73.8 │ 12e0738f-f722-43fe…  │
│ false                │ 2022-03-29 11:45:4…  │ … │         40.6 │        -74.1 │ fca1a7b4-6a20-4f86…  │
│ false                │ 2020-01-22 11:55:5…  │ … │         40.6 │        -74.1 │ abd59ea3-2dcf-4c6d…  │
│   ·                  │          ·           │ · │           ·  │          ·   │          ·           │
│   ·                  │          ·           │ · │           ·  │          ·   │          ·           │
│   ·                  │          ·           │ · │    

In [72]:
# copy to a Parquet file
sql = """
    COPY (SELECT * FROM new_york_city)
    TO 'data/nfip-claims-nyc.parquet'
"""

con.sql(sql)      

In [73]:
# copy to a CSV file
sql = """
    COPY (SELECT * FROM new_york_city)
    TO 'data/nfip-claims-nyc.csv'
"""

con.sql(sql)      

In [74]:
ls data/

FimaNfipClaims.parquet   nfip-claims-nyc.csv      nfip-claims-nyc.parquet


In [75]:
# preview size of file
!du -sh data/* | sort -rh

151M	data/FimaNfipClaims.parquet
 16M	data/nfip-claims-nyc.csv
3.4M	data/nfip-claims-nyc.parquet


In [76]:
# sanity check on exported Parquet file
sql = """
    SELECT *
    FROM read_parquet('data/nfip-claims-nyc.parquet')
    ORDER BY asOfDate DESC
    LIMIT 10;
"""

duckdb.sql(sql)

┌──────────────────────┬──────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ … │   latitude   │  longitude   │          id          │
│       boolean        │ timestamp with tim…  │   │ decimal(9,1) │ decimal(9,1) │         uuid         │
├──────────────────────┼──────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ false                │ 2025-03-10 11:48:3…  │ … │         40.6 │        -73.8 │ fa4d480a-8be8-4dda…  │
│ false                │ 2025-03-10 11:48:3…  │ … │         40.6 │        -73.8 │ 40e43084-68ea-439a…  │
│ false                │ 2025-03-10 11:48:3…  │ … │         40.6 │        -73.8 │ df70fbd5-c7cf-4e15…  │
│ false                │ 2025-03-10 10:43:5…  │ … │         40.6 │        -74.1 │ 3e4af504-2adb-4627…  │
│ false                │ 2025-03-10 10:43:5…  │ … │         40.6 │        -74.1 │ 43d69c61-7f50-4e8a…  │
│ false                │ 2025-03-10 06:17:1…  │ … │    

In [77]:
# sanity check on exported CSV file
sql = """
    SELECT *
    FROM read_csv('data/nfip-claims-nyc.csv')
    ORDER BY asOfDate DESC
    LIMIT 10;
"""

duckdb.sql(sql)

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────┬───────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ basementEnclosureC…  │ … │ latitude │ longitude │          id          │
│       boolean        │      timestamp       │        int64         │   │  double  │  double   │       varchar        │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────┼───────────┼──────────────────────┤
│ false                │ 2025-03-10 15:48:3…  │                    2 │ … │     40.6 │     -73.8 │ fa4d480a-8be8-4dda…  │
│ false                │ 2025-03-10 15:48:3…  │                    2 │ … │     40.6 │     -73.8 │ 40e43084-68ea-439a…  │
│ false                │ 2025-03-10 15:48:3…  │                    2 │ … │     40.6 │     -73.8 │ df70fbd5-c7cf-4e15…  │
│ false                │ 2025-03-10 14:43:5…  │                    2 │ … │     40.6 │     -74.1 │ 3e4af504-2adb-4627…  │
│ false                │ 2025-03

In [78]:
# sanity check on exported Parquet file
sql = """
    SELECT COUNT(*) AS count
    FROM read_parquet('data/nfip-claims-nyc.parquet');
"""

duckdb.sql(sql)

┌───────┐
│ count │
│ int64 │
├───────┤
│ 43978 │
└───────┘

In [79]:
# sanity check on exported CSV file
sql = """
    SELECT COUNT(*) AS count
    FROM read_csv('data/nfip-claims-nyc.csv');
"""

duckdb.sql(sql)

┌───────┐
│ count │
│ int64 │
├───────┤
│ 43978 │
└───────┘